<a target="_blank" href="https://colab.research.google.com/github/segmed/gemma_mistral/blob/main/notebooks/prompts_example_llm.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
!pip install -qq "transformers==4.38.0" --upgrade
!pip -qq install bitsandbytes accelerate datasets
!pip -qq install "torch>=2.1.1" # https://huggingface.co/google/gemma-7b/discussions/17

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━

In [22]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True)
model_name = "google/gemma-7b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             quantization_config=quantization_config)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [23]:
from datasets import load_dataset
dataset = load_dataset('medmcqa', split='train').shuffle(seed=42)
dataset

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
    num_rows: 182822
})

In [24]:
for question in dataset['question'][0:2]:
    chat = [
      { "role": "user", "content": question },
    ]
    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**input_ids, max_new_tokens=32, do_sample=True, top_k=0, num_return_sequences=1, temperature=0.01)
    display(tokenizer.decode(outputs[0]))

'<bos><start_of_turn>user\nOPV can be used if vaccine l monitor is showing?<end_of_turn>\n<start_of_turn>model\nSure, here is the answer to the question:\n\nOPV can be used if vaccine l monitor is showing, but it is recommended to consult with a healthcare'

'<bos><start_of_turn>user\nAverage gain of height in first year ?<end_of_turn>\n<start_of_turn>model\nThe average gain of height in the first year is around 2-3 inches.<eos>'

In [25]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True)
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             quantization_config=quantization_config)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [26]:
for question in dataset['question'][0:2]:
    chat = [
      { "role": "user", "content": question },
    ]
    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**input_ids, max_new_tokens=64, do_sample=True, top_k=0, num_return_sequences=1, temperature=0.01, eos_token_id=tokenizer.eos_token_id).cuda()
    display(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s><s> [INST] OPV can be used if vaccine l monitor is showing? [/INST] OPV (Oral Polio Vaccine) is a type of polio vaccine that is given orally, while a vaccine monitor is a person or organization responsible for overseeing the development, testing, and implementation of vaccines.\n\nOPV can be used even if there is a vaccine monitor in'

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s><s> [INST] Average gain of height in first year ? [/INST] The average height growth in the first year of life varies greatly depending on the age at which you consider the first year to be. In infants, the average height growth in the first year is around 2.5 inches (6.4 cm) for boys and 2.3 inches (5.8 cm'